In [44]:
!pip install -q --upgrade tensorflow==2.19.0
!pip install -q --upgrade matplotlib gradio tensorflow-datasets torch torchvision diffusers transformers accelerate safetensors

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from PIL import Image
import gradio as gr


import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import gradio as gr

IMG_SIZE = 128
BATCH_SIZE = 16
dataset_dir = "dataset"

# Dataset
train_ds = image_dataset_from_directory(
    dataset_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=0.2,
    subset='training',
    seed=123
)
val_ds = image_dataset_from_directory(
    dataset_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=0.2,
    subset='validation',
    seed=123
)
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
AUTOTUNE = tf.data.AUTOTUNE
normalizer = Rescaling(1./255)
train_ds = train_ds.map(lambda x,y: (normalizer(x), y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.map(lambda x,y: (normalizer(x), y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

# Jednostavan CNN
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE,3))
x = tf.keras.layers.Conv2D(16,3,activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(32,3,activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64,3,activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Grad-CAM
# AUTOMATSKO PRONALAŽENJE POSLEDNJEG Conv2D sloja
last_conv_layer_name = None
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name

print("Last conv layer:", last_conv_layer_name)


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = Model([model.inputs],[model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap,0)/(tf.reduce_max(heatmap)+1e-10)
    heatmap = np.uint8(255*heatmap)
    heatmap = Image.fromarray(heatmap).resize((IMG_SIZE, IMG_SIZE), Image.BILINEAR)
    return np.array(heatmap)/255.0

def overlay_heatmap(img, heatmap, alpha=0.4):
    img = np.array(img)/255.0
    if img.ndim==2: img = np.stack([img]*3,axis=-1)
    cmap = plt.cm.jet
    heatmap_color = cmap(heatmap)[:,:,:3]
    overlay = heatmap_color*alpha + img*(1-alpha)
    overlay = np.clip(overlay,0,1)
    return np.uint8(overlay*255)

def preprocess(img):
    if img.mode != "RGB": img = img.convert("RGB")
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img_array = np.expand_dims(np.array(img)/255.0,0)
    return img_array

def predict_and_explain(img):
    x = preprocess(img)
    preds = model.predict(x, verbose=0)
    idx = int(np.argmax(preds[0]))
    conf = float(np.max(preds[0]))
    heatmap = make_gradcam_heatmap(x, model, last_conv_layer_name, idx)
    overlay = overlay_heatmap(np.squeeze(x,0), heatmap)
    return {class_names[idx]: round(conf,3)}, overlay

iface = gr.Interface(fn=predict_and_explain,
                     inputs=gr.Image(type="pil"),
                     outputs=[gr.Label(num_top_classes=2), gr.Image(type="numpy")],
                     title="Real vs AI Image Classifier + Grad-CAM")
iface.launch(share=True)


Found 78 files belonging to 2 classes.
Using 63 files for training.
Found 78 files belonging to 2 classes.
Using 15 files for validation.


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_29 (InputLayer)     │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_19     │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,714 (92.63 KB)

 Trainable params: 23,714 (92.63 KB)

 Non-trainable params: 0 (0.00 B)

Last conv layer: conv2d_29
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a0f884971474da9f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
